# 1. Summarize the paper on BPE

Byte pair encoding is an algorithm for word segmentation, allowing for the representation of an open vocabulary through a fixed-size vocabulary of variable length. It's a versatile model utilized for many word segmentation strategies.
From the conclusion, , they were able to show that their variant of BPE that is capable of encoding open vocabs with a compact vocab filled with subwords.
BPE is a data comrpession technique that iteratively replaces the most frequenct pair of bytes in a sequence with a single, unused byte. (merging characters)
To learn the BPE operations, an example was given on learned BPE operations utilizing the following 4 words.

* low
* lower
* newest
* widest

Looking at lower, it would segment into low-er. 
After BPE merging operations, new words learned are

* low
* lowest
* newer
* wider

Research questions are as follows, can they improve translation of rare and unseen words by representing them in subword units?

Experimentals utilize a training set for translation from English -> German and English -> russian from WMT 2015. They compared BPE performance versus baseline methods.

Metrics they utilized were BLEU, CHRF3, and unigram F1

At the end of their evaluation, BPE met their goal of being an open vocabulary to obtain segmentation with no unknown symbols. Subword models improve over the baseline model, especially in the case for rare and unknown words as showcased from the figures in the paper

# 2. In regular expressions, what does \d, \D, \w, \W, \s, \S, {n}, {n,m}, {n,}, {,m} mean?

* \d - any digit
* \D - any non digit
* \w - any alphanumeric/underscore
* \W - a non alphanumeric
* \s - whitespace (space or tab)
* \S - non white space
* {n} - exactly n occurences of previous character or expression
* {n,m} - from n to m occurences of previous character or expression
* {n,} - at least n occurences of previous character or expression
* {,m} at most m occurences of previous character or expression

# 3. Create a Python based tokenizer in NLTK that replaces contractions (I’m, You’re, didn’t) into the expanded forms (I am, You are, did not).

In this tokenizer, you can split words by spaces (' \t\n').


In [1]:
!pip install contractions

  Obtaining dependency information for contractions from https://files.pythonhosted.org/packages/bb/e4/725241b788963b460ce0118bfd5c505dd3d1bdd020ee740f9f39044ed4a7/contractions-0.1.73-py2.py3-none-any.whl.metadata
  Obtaining dependency information for textsearch>=0.0.21 from https://files.pythonhosted.org/packages/e2/0f/6f08dd89e9d71380a369b1f5b6c97a32d62fc9cfacc1c5b8329505b9e495/textsearch-0.0.24-py2.py3-none-any.whl.metadata
  Obtaining dependency information for anyascii from https://files.pythonhosted.org/packages/4f/7b/a9a747e0632271d855da379532b05a62c58e979813814a57fa3b3afeb3a4/anyascii-0.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for pyahocorasick from https://files.pythonhosted.org/packages/36/76/d83c60ec7a202cbfeffaa9649d0fee6ddcb974622e411b86211ff3572549/pyahocorasick-2.1.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/289.9 kB ? eta -:--:--
   -------- ------------------------------- 61.4/289.9 kB 1.7 MB/s eta 0:00:

In [7]:
import nltk
import re
import contractions

def contraction_tokenizer(text):
    expanded_text = contractions.fix(text)
    
    tokens = nltk.word_tokenize(expanded_text)
    return tokens

In [8]:
sample_text = "I'm here but you're not. They've gone to school and won't be back until later."
expanded_txt = contraction_tokenizer(sample_text)
print(expanded_txt)

['I', 'am', 'here', 'but', 'you', 'are', 'not', '.', 'They', 'have', 'gone', 'to', 'school', 'and', 'will', 'not', 'be', 'back', 'until', 'later', '.']


# 4. Implement the BPE algorithm with the following interface

Your normalization should expand the contractions you implemented in problem 3.

In [17]:
import contractions
from collections import Counter, defaultdict
import re

class Tokenizer:
    def __init__(self, vocab_size):
        self.vocab_size = vocab_size
        self.vocab = {"<unknown>": 0, "the": 1}
        self.t2s = ["<unknown>", "the"]
        self.bpe_rules = []
    
    def normalize(self, text):
        # Expand contractions
        expanded_text = contractions.fix(text)
        tokens = nltk.word_tokenize(expanded_text)
        return tokens
    
    def train(self, corpus):
        # Tokenize and normalize the corpus
        token_freq = Counter()
        for word in corpus:
            normalized_word = self.normalize(word)
            token_freq.update(normalized_word)

        vocab = set(token_freq.keys())
        while len(vocab) < self.vocab_size:
            pairs = Counter()
            for word, freq in token_freq.items():
                symbols = word.split()
                for i in range(len(symbols) - 1):
                    pairs[(symbols[i], symbols[i + 1])] += freq
            
            if not pairs:
                break
            
            best_pair = max(pairs, key=pairs.get)
            if pairs[best_pair] < 2:
                break
            
            self.bpe_rules.append(best_pair)
            new_token = ''.join(best_pair)
            vocab.add(new_token)
            replacements = {re.escape(' '.join(best_pair)): new_token}
            pattern = re.compile("|".join(replacements.keys()))
            token_freq = Counter({pattern.sub(lambda m: replacements[re.escape(m.group(0))], word): freq for word, freq in token_freq.items()})
        
        # Update vocab and t2s
        for token in vocab:
            if token not in self.vocab:
                self.vocab[token] = len(self.vocab)
                self.t2s.append(token)

    def encode(self, text):
        normalized_tokens = self.normalize(text)
        return [self.vocab.get(token, 0) for token in normalized_tokens]
    
    def decode(self, indices):
        return ' '.join(self.t2s[index] for index in indices)

# Usage
from nltk.corpus import brown

corpus = brown.words()[:20000] #limited for speed, change or remove for bigger range
tokenizer = Tokenizer(5000)
tokenizer.train(corpus)

encoded = tokenizer.encode("I'm here but you're not. They've gone to school and won't be back until later.")
decoded = tokenizer.decode(encoded)

print("Encoded:", encoded)
print("Decoded:", decoded)


Encoded: [3667, 758, 1937, 4121, 2468, 457, 1148, 1526, 2465, 4103, 2337, 2635, 290, 1630, 1092, 1148, 1184, 1166, 2850, 2444, 1526]
Decoded: I am here but you are not . They have gone to school and will not be back until later .


# 5. Using the package https://www.nltk.org/api/nltk.chat.html#module-nltk.chat, you will create a regular expression based chatbot that answers the following questions.  

    a. What’s the temperature today? (check for the temperature at weather channel)
    b. What’s my zip code? (based on the person’s location)
    c. How much is $19.99 in <currency>? (You can search Google for “exchange rate between us and <currency>”)
    d. What’s the definition of <word>? (you will need to look for the definition at Wikipedia or dictionary)
    
The chatbot structure from NLTK uses static string responses, you have to modify it (as in the example below) to allow for functional objects that can parse the web, for example.

Note that you will need to process HTML to create the answers, and you will use Beautiful Soup to do that. 

https://www.crummy.com/software/BeautifulSoup/bs4/doc/index.html?highlight=select

In [4]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from bs4 import BeautifulSoup



In [2]:
from nltk.chat.util import Chat, reflections

import random

temperature = 84

def get_temp():
    global temperature
    temperature += 1
    if temperature > 85:
        return 'It is hot today.'
    return f'The temperature is {temperature}.'

responses = (
    (
        r"(hello(.*))|(good [a-zA-Z]+)",
        (
            get_temp,
        ),
    ),
)

def respond(self, str):
    """
    Generate a response to the user input.
    :type str: str
    :param str: The string to be mapped
    :rtype: str
    """

    # check each pattern
    for (pattern, response) in self._pairs:
        match = pattern.match(str)

        # did the pattern match?
        if match:
            resp = random.choice(response)  # pick a random response
            resp = resp()
            resp = self._wildcards(resp, match)  # process wildcards

            # fix munged punctuation at the end
            if resp[-2:] == "?.":
                resp = resp[:-2] + "."
            if resp[-2:] == "??":
                resp = resp[:-2] + "?"
            return resp

chatbot = Chat(responses, reflections)
Chat.respond = respond

def chat():
    print("*" * 75)
    print("Chatbot!".center(75))
    print("*" * 75)
    print("Welcome.")

    chatbot.converse()
    
chat()

***************************************************************************
                                  Chatbot!                                 
***************************************************************************
Welcome.


KeyboardInterrupt: Interrupted by user

In [51]:
from nltk.chat.util import Chat, reflections
import random
import requests
from bs4 import BeautifulSoup

api_key = 'insertapikey'

# Define functions for each question response
def get_temperature():
    # Fetch weather from a weather site
    response = requests.get("https://weather.com/weather/today/")
    soup = BeautifulSoup(response.text, 'lxml')
    temp = soup.find('span', class_='CurrentConditions--tempValue--MHmYY').text
    return f"The temperature today is {temp}."

def get_zip_code():
    # Use the IPinfo API to get location data by IP address
    response = requests.get(f"https://ipinfo.io?token={api_key}")
    data = response.json()
    zip_code = get_zip_code_by_ip(api_key)
    return f"Your ZIP code based on your current IP is {zip_code}."

def convert_currency(currency):
    # Fetch conversion rate 
    response = requests.get(f"https://www.x-rates.com/calculator/?from=USD&to={currency}&amount=19.99")
    soup = BeautifulSoup(response.text, 'lxml')
    result = soup.find('span', class_='ccOutputTrail').previous_sibling.text
    return f"$19.99 in {currency.upper()} is approximately {result}."

def get_definition(word):
    # Fetch word definition from a dictionary site
    response = requests.get(f"https://www.dictionary.com/browse/{word}")
    soup = BeautifulSoup(response.text, 'lxml')
    definition = soup.find('div', class_='NZKOFkdkcvYgD3lqOIJw').text
    return f"The definition of {word} is: {definition}"

# Customize the Chat class response method
def custom_respond(self, user_input):
    for (pattern, responses) in self._pairs:
        match = re.match(pattern, user_input)

        if match:
            resp = random.choice(responses)  # pick a random response
            if callable(resp):
                resp = resp(*match.groups())
            return resp

    return "I'm not sure how to respond to that."

# Define pairs for questions and respective functions
pairs = (
    (r"[wW]hat.s the temperature today\??", [get_temperature]),
    (r"[wW]hat.s my zip code\??", [get_zip_code]),
    (r"[hH]ow much is \$19\.99 in ([a-zA-Z]+)\?", [convert_currency]),
    (r"[wW]hat.s the definition of (\w+)\?", [get_definition]),
)

# Override the Chat class with custom respond method
class CustomChat(Chat):
    def respond(self, user_input):
        return custom_respond(self, user_input)

#run the chatbot
chatbot = CustomChat(pairs, reflections)

def chat():
    print("*" * 75)
    print("Chatbot!".center(75))
    print("*" * 75)
    print("Welcome to the chatbot. Ask me a question!")

    input("Press Enter to start...")

    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            print("Goodbye!")
            break
        response = chatbot.respond(user_input)
        print("Chatbot:", response)

chat()


***************************************************************************
                             Enhanced Chatbot!                             
***************************************************************************
Welcome to the enhanced chatbot. Ask me a question!


Press Enter to start... What's the temperature today?
You:  What's the temperature today?


Chatbot: The temperature today is 59°.


You:  What's my zip code?


Chatbot: Your ZIP code based on your current IP is 95103.


...

You:  What's the definition of cockatrice?


Chatbot: The definition of cockatrice is:   a legendary monster with a deadly glance, supposedly hatched by a serpent from the egg of a rooster, and commonly represented with the head, legs, and wings of a rooster and the body and tail of a serpent. Compare basilisk ( def 1 ).


You:  Thanks


Chatbot: I'm not sure how to respond to that.


KeyboardInterrupt: Interrupted by user

In [29]:
#TEST CODE
response = requests.get("https://weather.com/weather/today/")
soup = BeautifulSoup(response.text, 'lxml')
print(soup.prettify()[:2000])  # Print only the first 2000 characters

<!DOCTYPE html>
<html dir="ltr" lang="en-US">
 <head>
  <meta charset="utf-8" data-react-helmet="true"/>
  <meta content="width=device-width, initial-scale=1, viewport-fit=cover" data-react-helmet="true" name="viewport"/>
  <meta content="max-image-preview:large" data-react-helmet="true" name="robots"/>
  <meta content="index, follow" data-react-helmet="true" name="robots"/>
  <meta content="origin" data-react-helmet="true" name="referrer"/>
  <meta content="Today’s and tonight’s San Jose, CA weather forecast, weather conditions and Doppler radar from The Weather Channel and Weather.com" data-react-helmet="true" name="description"/>
  <meta content="#ffffff" data-react-helmet="true" name="msapplication-TileColor"/>
  <meta content="/daybreak-today/assets/ms-icon-144x144.d353af.png" data-react-helmet="true" name="msapplication-TileImage"/>
  <meta content="#ffffff" data-react-helmet="true" name="theme-color"/>
  <meta content="app-id=295646461" data-react-helmet="true" name="apple-itune

In [31]:
#TEST CODE TO FIND TEMPERATURE
temp = soup.find('span', class_='CurrentConditions--tempValue--MHmYY')  
if temp:
    print("Temperature element found:", temp.text)
else:
    print("Temperature element not found.")

Temperature element found: 60°


In [43]:
#TEST CODE TO FIND ZIPCODE
from bs4 import BeautifulSoup
import requests

url = "https://www.mapdevelopers.com/what-is-my-zip-code.php"

# Make a request to the webpage
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')


zip_code = soup.find('div', class_='col-sm-4 col-md-4')
if zip_code:
    print("Zip code element found:", zip_code.text)
else:
    print("Zip code element not found.")

Zip code element found: 


In [47]:
#SECONDARY TEST CODE TO FIND ZIP CODE
import requests

def get_zip_code_by_ip(api_key):
    # Use the IPinfo API to get location data by IP address
    response = requests.get(f"https://ipinfo.io?token={api_key}")
    data = response.json()
    return data.get('postal')  # 'postal' is the key for the ZIP code in the IPinfo API response

api_key = '49b85054de195f'  # Replace with your actual API key
zip_code = get_zip_code_by_ip(api_key)
print("ZIP Code:", zip_code)

ZIP Code: 95103


# 6. Implement the spell checker for cell phones

Have you ever tried to type in something quickly and because the keyboard size in the cell phone is much smaller than your finger, it typing in the neighboring letters?

   a) You will implement the spell checker from the site: https://norvig.com/spell-correct.html

   b) You will change your code to consider that replacements would only occur to neighboring words. For example, in the picture, the letter 'u' can be replaced by 'y' or 'i'.

![Keyboard iPhone](keyboard.png)

In [8]:
keyboard_neighbors = {
    'a': 's',
    'b': 'vn',
    'c': 'xv',
    'd': 'sf',
    'e': 'wr',
    'f': 'dg',
    'g': 'fh',
    'h': 'gj',
    'i': 'uo',
    'j': 'hk',
    'k': 'jl',
    'l': 'k',
    'm': 'n',
    'n': 'bm',
    'o': 'ip',
    'p': 'o',
    'q': 'w',
    'r': 'et',
    's': 'da',
    't': 'ry',
    'u': 'yi',
    'v': 'cb',
    'w': 'qe',
    'x': 'zc',
    'y': 'tu',
    'z': 'x'
}

In [80]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    #Probability
    return WORDS[word] / N

def correction(word): 
    #Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    #Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    #The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    #All edits that are one edit away 
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in keyboard_neighbors.get(R[0], '')]
    inserts = [L + c + R for L, R in splits for c in (keyboard_neighbors.get(L[-1], '') if L else '') + (keyboard_neighbors.get(R[0], '') if R else '')]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    #All edits that are two edits away 
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [85]:
correction('vorrect')

'correct'

In [86]:
correction('cortect')

'correct'

In [87]:
correction('korrect')

'korrect'

In [88]:
correction('vorrecy')

'correct'

In [93]:
correction('cprrwct')

'correct'

# 7. Implement the Weighted Mininum Edit Distance

In this problem, you will implement the weighted minimum edit distance algorithm of slide 94.

You will consider the following.

- delete and insertion cost is 1
- substitution cost is 1 if it is in adjacent in the keyboard, like in problem 6.b
- substitution cost is 2 if it is below or above, or two characters to the right or left (for example, in the example of 6.b, replacing a 'u' by a 't' or 'o' would have a cost of 2
- substitution cost is infinity if that does not apply

Run the algorithm for the two words: 

- 'caft' vs 'cat' 
- 'coffee' vs 'voffrt'

In [9]:
def keyboard_substitution_cost(a,b):
    #Dictionary of neighboring keys    
    keyboard_neighbors = {
        'a': 's',
        'b': 'vn',
        'c': 'xv',
        'd': 'sf',
        'e': 'wr',
        'f': 'dg',
        'g': 'fh',
        'h': 'gj',
        'i': 'uo',
        'j': 'hk',
        'k': 'jl',
        'l': 'k',
        'm': 'n',
        'n': 'bm',
        'o': 'ip',
        'p': 'o',
        'q': 'w',
        'r': 'et',
        's': 'da',
        't': 'ry',
        'u': 'yi',
        'v': 'cb',
        'w': 'qe',
        'x': 'zc',
        'y': 'tu',
        'z': 'x'
    }
    #dictionary of keys 2 away or up and down
    expanded_keyboard_neighbors = {
        'q': 'ae', 'w': 'asr', 'e': 'qtsd', 'r': 'wydf', 't': 'fgeu',
        'y': 'ghri', 'u': 'hjto', 'i': 'jkyp', 'o': 'kly', 'p': 'il',
        'a': 'zqd', 's': 'zxwef', 'd': 'xcerag', 'f': 'cvrtsh', 'g': 'vbtydj',
        'h': 'bnyufk', 'j': 'nmuigl', 'k': 'mioh', 'l': 'opj',
        'z': 'asc', 'x': 'sdv', 'c': 'dfzb', 'v': 'fgxn', 'b': 'cmgh',
        'n': 'hjv', 'm': 'jkb'
    }
    
    if b in keyboard_neighbors.get(a, ''):
        return 1
    elif b in expanded_keyboard_neighbors.get(a, ''):
        return 2
    else:
        return float('inf')

In [11]:
def min_edit_distance(source, target, del_cost, ins_cost, sub_cost):
    n = len(source)
    m = len(target)
    
    # Create the distance matrix D
    D = [[0] * (m + 1) for _ in range(n + 1)]
    
    # initialization the zeroth row and column is the distance from the empty string
    for i in range(1, n + 1):
        D[i][0] = D[i - 1][0] + del_cost(source[i - 1])
    for j in range(1, m + 1):
        D[0][j] = D[0][j - 1] + ins_cost(target[j - 1])
    
    # recurrence relation:
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            deletion = D[i - 1][j] + del_cost(source[i - 1])
            insertion = D[i][j - 1] + ins_cost(target[j - 1])
            substitution = D[i - 1][j - 1] + sub_cost(source[i - 1], target[j - 1])
            D[i][j] = min(deletion, insertion, substitution)
    
    # Termination
    return D[n][m]

def del_cost(char):
    return 1  # Cost of deletion

def ins_cost(char):
    return 1  # Cost of insertion

def sub_cost(s_char, t_char):
    if s_char == t_char:
        return 0  # No cost if characters are the same
    else:
        return keyboard_substitution_cost(s_char, t_char)  # Cost of substitution

In [12]:
source = "caft"
target = "cat"
distance = min_edit_distance(source, target, del_cost, ins_cost, sub_cost)
print(f"Minimum edit distance: {distance}")

Minimum edit distance: 1


In [13]:
source = "voffrt"
target = "coffee"
distance = min_edit_distance(source, target, del_cost, ins_cost, sub_cost)
print(f"Minimum edit distance: {distance}")

Minimum edit distance: 4
